<a href="https://colab.research.google.com/github/RolakeOkans/pembrolizumab-hcc-response-modeling/blob/main/HPC_dataset_updated_11_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install adjustText

In [ ]:
# Response Data:From GSE242154.txt file, extract info for each sample and put into a dataframe
#This will enable me to see each sample, tissue type, roi, biomarker, response etc

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
import numpy as np

samples = []
current_sample = {}

with open('/content/GSE242154.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('^SAMPLE'):
            if current_sample:
                samples.append(current_sample)
            current_sample = {'Sample_ID': line.split('=')[1].strip(),
                              'Sample_title': '',
                              'Sample_geo_accession': ''}
        elif line.startswith('!Sample_title'):
            current_sample['Sample_title'] = line.split('=')[1].strip()
        elif line.startswith('!Sample_geo_accession'):
            current_sample['Sample_geo_accession'] = line.split('=')[1].strip()
        elif line.startswith('!Sample_characteristics_ch1'):
            # split by first colon to get key/value
            key_value = line.split('=')[1].strip().split(':', 1)
            if len(key_value) == 2:
                key = key_value[0].strip().replace(" ", "_")  # replace spaces with _
                value = key_value[1].strip()
                current_sample[key] = value

# Add last sample
if current_sample:
    samples.append(current_sample)

# Convert to DataFrame
df = pd.DataFrame(samples)


# Save to CSV
df.to_csv('samples_characteristics.csv', index=False)


In [ ]:
#Gene Expression Data: From GEO series_matrix files, get gene expression data for each sample
file_path = '/content/GSE242154_series_matrix.txt'
gene_exp_df = pd.read_csv(file_path, sep='\t', header=None, comment='!')  # ignore comment lines starting with !

# 2. Peek at the first few rows
print(gene_exp_df.head())

# 3. Get all column names (if they exist)
print("Columns:", gene_exp_df.columns.tolist())

# 4. If the first row contains the column names, reload with header
gene_exp_df = pd.read_csv(file_path, sep='\t', header=0, comment='!')
print("Columns now:", gene_exp_df.columns.tolist())

# 5. Optional: see number of rows and columns
print("Shape:", gene_exp_df.shape)

In [ ]:
#combine Response Data with Gene Expression Data

# Load metadata
metadata_df = pd.read_csv('samples_characteristics.csv')

# Check metadata sample IDs
print(metadata_df['Sample_ID'].head())

# Load expression data (series matrix), assuming first column = Protein, rest = samples
expr_file = '/content/GSE242154_series_matrix.txt'

# Read expression manually because comment lines interfere
expr_lines = []
with open(expr_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith('!') or line.startswith('^'):
            continue
        expr_lines.append(line.split('\t'))

# First column = protein names
protein_names = [row[0] for row in expr_lines]
sample_values = [row[1:] for row in expr_lines]

# Samples in the same order as they appear in the file
sample_ids = [col.strip() for col in expr_lines[0][1:]]

# Create expression DataFrame
expr_df = pd.DataFrame(sample_values, columns=metadata_df['Sample_ID'], index=protein_names).T
expr_df.index.name = 'Sample_ID'

# Merge metadata with expression data
combined_df = expr_df.merge(metadata_df, on='Sample_ID', how='left')

# Save final combined table
combined_df.to_csv('combined_expression_metadata.csv')

print("Combined shape:", combined_df.shape)
print(combined_df.head())


In [ ]:
# -------------------------------
# STEP 4: Quality Control Checks
# -------------------------------

print("\n===== QC CHECKS =====")

#Metadata integrity
print("\n--- Missing values per column ---")
print(combined_df.isnull().sum())

print("\n--- Unique Sample IDs ---")
print("Unique IDs:", combined_df['Sample_ID'].nunique())
print("Total rows:", combined_df.shape[0])

# Count samples per metadata category (if present)
for col in ['tissue_type', 'roi', 'biomarker', 'response']:
    if col in combined_df.columns:
        print(f"\n--- Counts for {col} ---")
        print(combined_df[col].value_counts())

In [ ]:
#Normalize protein expression using quantile normalization


# -------------------------------
# STEP 1: Load combined data
# -------------------------------
combined_df = pd.read_csv("combined_expression_metadata.csv")

# -------------------------------
# STEP 2: Clean column names
# -------------------------------
combined_df.columns = [c.strip().replace('"','') for c in combined_df.columns]

# Explicit metadata columns (these will not be normalized)
metadata_cols = [
    'Sample_ID','Sample_title','Sample_geo_accession','tissue_type',
    'region_of_interest_(roi)_number','segment_type',
    'positive_immuno-fluorescent_morphology_marker',
    'radiographic_objective_response'
]

# -------------------------------
# STEP 3: Define proteins to normalize
# -------------------------------
proteins_to_normalize = [
    "4-1BB","ARG1","B7-H3","Bcl-2","Beta-2-microglobulin","CD11c","CD127",
    "CD14","CD163","CD20","CD25","CD27","CD3","CD34","CD4","CD40","CD44",
    "CD45","CD45RO","CD56","CD66b","CD68","CD8","CD80","CTLA4","EpCAM",
    "ER-alpha","FAP-alpha","Fibronectin","FOXP3","GAPDH","GITR","GZMB",
    "Her2","Histone H3","HLA-DR","ICOS","IDO1","Ki-67","LAG3","MART1",
    "Ms IgG1","Ms IgG2a","NY-ESO-1","OX40L","PanCk","PD-1","PD-L1","PD-L2",
    "PR","PTEN","Rb IgG","S100B","S6","SMA","STING","Tim-3","VISTA"
]

# Clean protein names
proteins_to_normalize = [p.strip().replace('"','') for p in proteins_to_normalize]

# Keep only proteins that exist in the dataset
expr_cols = [c for c in proteins_to_normalize if c in combined_df.columns]
print("Proteins to normalize found in dataset:", expr_cols)

# -------------------------------
# STEP 4: Extract protein expression data and ensure numeric
# -------------------------------
expr_df = combined_df[expr_cols].apply(pd.to_numeric, errors='coerce')

# Check for all-NaN columns
expr_df = expr_df.dropna(axis=1, how='all')

# -------------------------------
# STEP 5: Quantile normalization
# -------------------------------
def quantile_normalize(df):
    """Perform quantile normalization on a DataFrame (samples x proteins)."""
    # rank the values
    sorted_df = np.sort(df.values, axis=0)
    # compute row-wise mean
    mean_ranks = np.mean(sorted_df, axis=1)
    # get the ranks for each column
    ranks = df.stack().groupby(df.rank(method='first').stack().astype(int)).mean()
    # map ranks to mean values
    rank_index = df.rank(method='min').stack().astype(int)
    normalized_values = rank_index.map(lambda x: mean_ranks[x-1])
    normalized_df = normalized_values.unstack()
    return normalized_df

expr_norm_df = quantile_normalize(expr_df)

# -------------------------------
# STEP 6: Merge normalized proteins back with metadata
# -------------------------------
final_df = combined_df.copy()
for col in expr_norm_df.columns:
    final_df[col] = expr_norm_df[col]

# -------------------------------
# STEP 7: Save final CSV
# -------------------------------
final_df.to_csv("combined_expression_metadata_quantile_normalized.csv", index=False)
print("✅ Quantile normalization completed and saved")


In [ ]:
# Look for differentially expressed proteins and visualize them using volcano plots
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# -------------------------------
# STEP 1: Load normalized data
# -------------------------------
df = pd.read_csv("combined_expression_metadata_quantile_normalized.csv")

# Clean response column
df['radiographic_objective_response'] = df['radiographic_objective_response'].astype(str).str.strip()

# -------------------------------
# STEP 2: Define groups
# -------------------------------
responders = df[df['radiographic_objective_response'].isin(['complete response (CR)', 'partial response (PR)'])]
non_responders = df[df['radiographic_objective_response'].isin(['progressive disease (PD)', 'stable disease (SD)'])]

# Metadata columns
metadata_cols = [
    'Unnamed: 0', 'Sample_ID', 'ID_REF', 'Sample_title', 'Sample_geo_accession',
    'patient_id', 'tissue_type', 'region_of_interest_(roi)_number', 'segment_type',
    'positive_immuno-fluorescent_morphology_marker', 'radiographic_objective_response'
]
protein_cols = [c for c in df.columns if c not in metadata_cols]

# -------------------------------
# OPTIONAL: Top 15 proteins to label on volcano plots
# IMPORTANT: These names must match exactly how they appear in results_df["Protein"]
# If your column names differ (e.g., "HLA-DRA" vs "HLA-DR"), edit accordingly.
# -------------------------------
top15_proteins = [
    "Her2", "MART1", "PTEN", "NY-ESO-1", "CD20",
    "PR", "CD56", "HLA-DR", "S6", "ER-alpha",
    "GAPDH", "IDO1", "PD-1", "GZMB", "LAG3"
]

# -------------------------------
# STEP 3: Differential Expression
# -------------------------------
results = []
for protein in protein_cols:
    vals_resp = pd.to_numeric(responders[protein], errors='coerce').dropna()
    vals_nonresp = pd.to_numeric(non_responders[protein], errors='coerce').dropna()

    if len(vals_resp) < 2 or len(vals_nonresp) < 2:
        continue

    mean_resp = vals_resp.mean()
    mean_nonresp = vals_nonresp.mean()

    if mean_nonresp == 0:  # avoid division by zero
        continue

    fold_change = mean_resp / mean_nonresp
    log2_fc = np.log2(fold_change)

    t_stat, p_val = ttest_ind(vals_resp, vals_nonresp, equal_var=False)

    results.append({
        "Protein": protein,
        "Mean_Responders": mean_resp,
        "Mean_NonResponders": mean_nonresp,
        "FoldChange": fold_change,
        "log2FC": log2_fc,
        "p_value": p_val
    })

results_df = pd.DataFrame(results)

# Adjust p-values (FDR BH)
results_df["adj_p_value"] = multipletests(results_df["p_value"], method="fdr_bh")[1]

# -------------------------------
# STEP 4: Test thresholds + Volcano plots
# -------------------------------
pval_thresh = 0.05
fc_thresholds = {
    "2-fold": 1.0,             # |log2FC| >= 1
    "1.5-fold": np.log2(1.5),  # |log2FC| >= 0.585
    "1.2-fold": np.log2(1.2)   # |log2FC| >= 0.263
}

summary = []

for label, fc_thr in fc_thresholds.items():
    # Subset DE proteins for this threshold
    subset = results_df[
        (results_df["adj_p_value"] < pval_thresh) &
        (abs(results_df["log2FC"]) >= fc_thr)
    ]
    subset.to_csv(f"DE_results_{label}_p0.05.csv", index=False)

    summary.append({
        "Threshold": label,
        "log2FC_cutoff": round(fc_thr, 3),
        "n_DE": len(subset),
        "Up_in_Responders": (subset["log2FC"] > 0).sum(),
        "Up_in_NonResponders": (subset["log2FC"] < 0).sum()
    })

    # Mark significance (THIS IS YOUR ORIGINAL LOGIC)
    results_df["Significant"] = results_df.apply(
        lambda r: "Significant"
        if (r["adj_p_value"] < pval_thresh and abs(r["log2FC"]) >= fc_thr)
        else "Not significant",
        axis=1
    )

    # Volcano plot
    plt.figure(figsize=(12, 9))
    sns.scatterplot(
        data=results_df,
        x="log2FC",
        y=-np.log10(results_df["adj_p_value"]),
        hue="Significant",
        palette={"Significant": "red", "Not significant": "grey"},
        legend=True
    )
    handles, labels = plt.gca().get_legend_handles_labels()
    unique = dict(zip(labels, handles))
    plt.legend(unique.values(), unique.keys(), title="Significance")




    # Threshold lines
    plt.axhline(-np.log10(pval_thresh), color='black', linestyle='--')
    plt.axvline(fc_thr, color='green', linestyle='--')
    plt.axvline(-fc_thr, color='green', linestyle='--')

    # -------------------------------
    # LABEL TOP 15 PROTEINS (NO CHANGE TO SIGNIFICANCE)
    # -------------------------------
    texts = []
    # Only label proteins that exist in results_df and are in your top15 list
    label_df = results_df[results_df["Protein"].isin(top15_proteins)].copy()

    for _, row in label_df.iterrows():
        texts.append(
            plt.text(
                row["log2FC"],
                -np.log10(row["adj_p_value"]),
                row["Protein"],
                fontsize=9
            )
        )

    # Prevent label overlap
    adjust_text(
        texts,
        arrowprops=dict(arrowstyle='-', color='black', lw=0.5)
    )

    plt.title(f"Volcano Plot ({label} cutoff, adjusted p=0.05)")
    plt.xlabel("log2 Fold Change (Responder / Non-responder)")
    plt.ylabel("-log10(Adjusted p-value)")

    plt.tight_layout()
    plt.savefig(f"volcano_{label}_p0.05_top15_labeled.png", dpi=300)
    plt.close()

# Save summary
summary_df = pd.DataFrame(summary)
summary_df.to_csv("DE_summary_all_thresholds.csv", index=False)
print(summary_df)

# Optional sanity check: which of the top15 proteins were found in results_df?
found = [p for p in top15_proteins if p in results_df["Protein"].values]
missing = [p for p in top15_proteins if p not in results_df["Protein"].values]
print("\nTop15 proteins found in DE results:", found)
print("Top15 proteins missing (name mismatch?):", missing)


In [ ]:
#Look for differentially expressed genes and visualize them using volcano plots
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# -------------------------------
# STEP 1: Load normalized data
# -------------------------------
df = pd.read_csv("combined_expression_metadata_quantile_normalized.csv")

# Clean response column
df['radiographic_objective_response'] = df['radiographic_objective_response'].astype(str).str.strip()

# -------------------------------
# STEP 2: Define groups
# -------------------------------
responders = df[df['radiographic_objective_response'].isin(['complete response (CR)', 'partial response (PR)'])]
non_responders = df[df['radiographic_objective_response'].isin(['progressive disease (PD)', 'stable disease (SD)'])]

# Metadata columns
metadata_cols = [
    'Unnamed: 0', 'Sample_ID', 'ID_REF', 'Sample_title', 'Sample_geo_accession',
    'patient_id', 'tissue_type', 'region_of_interest_(roi)_number', 'segment_type',
    'positive_immuno-fluorescent_morphology_marker', 'radiographic_objective_response'
]
protein_cols = [c for c in df.columns if c not in metadata_cols]

# -------------------------------
# STEP 3: Differential Expression
# -------------------------------
results = []
for protein in protein_cols:
    vals_resp = pd.to_numeric(responders[protein], errors='coerce').dropna()
    vals_nonresp = pd.to_numeric(non_responders[protein], errors='coerce').dropna()

    if len(vals_resp) < 2 or len(vals_nonresp) < 2:
        continue

    mean_resp = vals_resp.mean()
    mean_nonresp = vals_nonresp.mean()

    if mean_nonresp == 0:  # avoid division by zero
        continue

    fold_change = mean_resp / mean_nonresp
    log2_fc = np.log2(fold_change)

    t_stat, p_val = ttest_ind(vals_resp, vals_nonresp, equal_var=False)

    results.append({
        "Protein": protein,
        "Mean_Responders": mean_resp,
        "Mean_NonResponders": mean_nonresp,
        "FoldChange": fold_change,
        "log2FC": log2_fc,
        "p_value": p_val
    })

results_df = pd.DataFrame(results)

# Adjust p-values (but p threshold remains fixed at 0.05)
results_df["adj_p_value"] = multipletests(results_df["p_value"], method="fdr_bh")[1]

# -------------------------------
# STEP 4: Test thresholds
# -------------------------------
# Keep p=0.05 fixed
pval_thresh = 0.05
fc_thresholds = {
    "2-fold" : 1.0,      # log2FC >= 1
    "1.5-fold": np.log2(1.5),  # log2FC >= 0.585
    "1.2-fold": np.log2(1.2)   # log2FC >= 0.263
}

summary = []

for label, fc_thr in fc_thresholds.items():
    subset = results_df[(results_df["adj_p_value"] < pval_thresh) & (abs(results_df["log2FC"]) >= fc_thr)]
    subset.to_csv(f"DE_results_{label}_p0.05.csv", index=False)

    summary.append({
        "Threshold": label,
        "log2FC_cutoff": round(fc_thr, 3),
        "n_DE": len(subset),
        "Up_in_Responders": (subset["log2FC"] > 0).sum(),
        "Up_in_NonResponders": (subset["log2FC"] < 0).sum()
    })

    # Volcano plot for each threshold
    results_df["Significant"] = results_df.apply(
        lambda r: "Significant" if (r["adj_p_value"] < pval_thresh and abs(r["log2FC"]) >= fc_thr)
                  else "Not significant", axis=1
    )

    plt.figure(figsize=(12,9))
    sns.scatterplot(
        data=results_df,
        x="log2FC", y=-np.log10(results_df["adj_p_value"]),
        hue="Significant",
        palette={"Significant":"red", "Not significant":"grey"},
        legend=True
    )

    # threshold lines
    plt.axhline(-np.log10(pval_thresh), color='black', linestyle='--')
    plt.axvline(fc_thr, color='green', linestyle='--')
    plt.axvline(-fc_thr, color='green', linestyle='--')

    plt.title(f"Volcano Plot ({label} cutoff, p=0.05)")
    plt.xlabel("log2 Fold Change (Responder / Non-responder)")
    plt.ylabel("-log10(Adjusted p-value)")

    plt.tight_layout()
    plt.savefig(f"volcano_{label}_p0.05.png", dpi=300)
    plt.close()

summary_df = pd.DataFrame(summary)
summary_df.to_csv("DE_summary_all_thresholds.csv", index=False)
print(summary_df)


In [ ]:
#Train a Random Forest Classifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve, auc,
    classification_report, confusion_matrix
)

from sklearn.ensemble import RandomForestClassifier


# ----------------------------------------------------------
# LOAD DATA
# ----------------------------------------------------------
data = pd.read_csv("combined_expression_metadata_quantile_normalized.csv")

data['Response_Group'] = data['radiographic_objective_response'].replace({
    'complete response (CR)': 1,
    'partial response (PR)': 1,
    'progressive disease (PD)': 0,
    'stable disease (SD)': 0
})


# ----------------------------------------------------------
# SELECT FEATURES / TARGET
# ----------------------------------------------------------
metadata_cols = [
    'Unnamed: 0','Sample_ID','ID_REF','Sample_title','Sample_geo_accession',
    'patient_id','tissue_type','region_of_interest_(roi)_number','segment_type',
    'positive_immuno-fluorescent_morphology_marker',
    'radiographic_objective_response','Response_Group'
]

X = data.drop(columns=metadata_cols)
y = data['Response_Group']


# ----------------------------------------------------------
# TRAIN / TEST SPLIT
# ----------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

print(f"TRAIN: {X_train.shape[0]}")
print(f"TEST:  {X_test.shape[0]}")


# ----------------------------------------------------------
# SCALE DATA
# ----------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# ----------------------------------------------------------
# RANDOM FOREST (THE ONE THAT WORKED)
# ----------------------------------------------------------
rf = RandomForestClassifier(
    n_estimators=500,
    random_state=42
)

rf.fit(X_train_scaled, y_train)

y_pred_rf = rf.predict(X_test_scaled)
y_prob_rf = rf.predict_proba(X_test_scaled)[:, 1]


# ----------------------------------------------------------
# EVALUATION
# ----------------------------------------------------------
print("\n===== RANDOM FOREST (ORIGINAL HIGH-SCORE MODEL) =====")
print("Accuracy:", round(accuracy_score(y_test, y_pred_rf), 3))
print("Precision:", round(precision_score(y_test, y_pred_rf), 3))
print("Recall:", round(recall_score(y_test, y_pred_rf), 3))
print("F1-score:", round(f1_score(y_test, y_pred_rf), 3))
print("ROC AUC:", round(roc_auc_score(y_test, y_prob_rf), 3))

print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred_rf))

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred_rf))


# ----------------------------------------------------------
# PLOT ROC
# ----------------------------------------------------------
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

plt.figure(figsize=(7,6))
plt.plot(fpr_rf, tpr_rf, color='blue', lw=2,
         label=f'Random Forest (AUC = {roc_auc_rf:.2f})')
plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Random Forest (Original)')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig("RF_original_highscore.png", dpi=300)
plt.show()


# ----------------------------------------------------------
# FEATURE IMPORTANCE
# ----------------------------------------------------------
rf_importance_df = pd.DataFrame({
    'Protein': X.columns,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 15 Important Proteins:")
print(rf_importance_df.head(15))

rf_importance_df.to_csv("RF_feature_importance_original.csv", index=False)


In [ ]:
# Select top 15 proteins
top15 = rf_importance_df.head(15)

# Reverse order for horizontal bar plot (largest on top)
top15 = top15.iloc[::-1]

plt.figure(figsize=(8, 6))
plt.barh(top15['Protein'], top15['Importance'])
plt.xlabel('Feature Importance')
plt.title('Top 15 Proteins Contributing to Random Forest Predictions')

plt.tight_layout()
plt.savefig("RF_top15_feature_importance.png", dpi=300)
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ----------------------------------------------------------
# PREDICTION SCORES FOR EACH PATIENT
# ----------------------------------------------------------

# Probability that a sample is a responder (class = 1)
prediction_scores = rf.predict_proba(X_test_scaled)[:, 1]

# Convert to dataframe with IDs
results_with_scores = pd.DataFrame({
    "Sample_ID": X_test.index,
    "True_Label": y_test.values,
    "Predicted_Label": y_pred_rf,
    "Prediction_Score": prediction_scores
})

# Sort by score (highest responders first)
results_with_scores = results_with_scores.sort_values(by="Prediction_Score", ascending=False)

print("\n===== PATIENT-LEVEL PREDICTION SCORES =====")
print(results_with_scores.head())

# Save to CSV
results_with_scores.to_csv("patient_prediction_scores_RF.csv", index=False)
print("\nSaved → patient_prediction_scores_RF.csv")


In [ ]:
import pandas as pd
import numpy as np

# ----------------------------------------------------------
# BUILD PREDICTION TABLE (DEFAULT 0.5)
# ----------------------------------------------------------
prediction_table = pd.DataFrame({
    "Sample_ID": data.loc[y_test.index, "Sample_ID"].values,
    "True_Label": y_test.values,
    "Predicted_Label": y_pred_rf,
    "Prediction_Score": np.round(y_prob_rf, 4)   # probability of being a responder
})

# Sort by score (optional)
prediction_table = prediction_table.sort_values("Prediction_Score", ascending=False)

# Save
prediction_table.to_csv("rf_prediction_table_default_threshold.csv", index=False)

print("Saved → rf_prediction_table_default_threshold.csv")

# Show first few rows
prediction_table.head()


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# ----------------------------------------------------------
# Your top 15 proteins EXACTLY as present in your dataset
# ----------------------------------------------------------
top15 = [
    "Her2","MART1","PTEN","NY-ESO-1","CD20","PR","CD56",
    "HLA-DR","S6","ER-alpha","GAPDH","IDO1","PD-1","GZMB","LAG3"
]

# ----------------------------------------------------------
# Load expression matrix + response groups
# ----------------------------------------------------------
expr = pd.read_csv("combined_expression_metadata_quantile_normalized.csv")

# Make response label
expr['Response_Group'] = expr['radiographic_objective_response'].replace({
    'complete response (CR)': 1,
    'partial response (PR)': 1,
    'progressive disease (PD)': 0,
    'stable disease (SD)': 0
})

# Split data
responders = expr[expr["Response_Group"] == 1]
nonresponders = expr[expr["Response_Group"] == 0]

results = []

# ----------------------------------------------------------
# Compute differential expression for each of the 15 proteins
# ----------------------------------------------------------
for protein in top15:

    if protein not in expr.columns:
        print(f"WARNING: {protein} not found in dataset")
        continue

    r_vals = responders[protein].astype(float)
    nr_vals = nonresponders[protein].astype(float)

    # Mean expression
    mean_r = r_vals.mean()
    mean_nr = nr_vals.mean()

    # log2 fold change
    log2fc = np.log2((mean_r + 1e-8) / (mean_nr + 1e-8))

    # Two-sample t-test
    t_stat, p_val = ttest_ind(r_vals, nr_vals, equal_var=False)

    results.append([protein, mean_r, mean_nr, log2fc, p_val])


# ----------------------------------------------------------
# Adjust p-values for multiple testing
# ----------------------------------------------------------
results_df = pd.DataFrame(results, columns=["Protein","Mean_R","Mean_NR","Log2FC","p_value"])
results_df["adj_p_value"] = multipletests(results_df["p_value"], method="fdr_bh")[1]

# Direction
results_df["Direction"] = results_df["Log2FC"].apply(
    lambda x: "Up in Responders" if x > 0 else "Up in Non-responders"
)

print(results_df)

results_df.to_csv("Top15_significance_test.csv", index=False)
print("\nSaved → Top15_significance_test.csv")


In [ ]:
results_df.head()

In [ ]:
from adjustText import adjust_text

# Load RF importance and get top 15 proteins
rf_importance_df = pd.read_csv("RF_feature_importance_original.csv")
top15_proteins = rf_importance_df.head(15)['Protein'].tolist()

# Thresholds
pval_thresh = 0.05
fc_thr = np.log2(1.2)  # 1.2-fold threshold

# Mark significance using YOUR column name: "Log2FC"
results_df["Significant"] = results_df.apply(
    lambda r: "Significant"
    if (r["adj_p_value"] < pval_thresh and abs(r["Log2FC"]) >= fc_thr)
    else "Not significant",
    axis=1
)

plt.figure(figsize=(12, 9))

sns.scatterplot(
    data=results_df,
    x="Log2FC",
    y=-np.log10(results_df["adj_p_value"]),
    hue="Significant",
    palette={"Significant": "red", "Not significant": "grey"},
    legend=True
)

# Threshold lines
plt.axhline(-np.log10(pval_thresh), color='black', linestyle='--')
plt.axvline(fc_thr, color='green', linestyle='--')
plt.axvline(-fc_thr, color='green', linestyle='--')

# Label top 15 proteins
texts = []
for _, row in results_df.iterrows():
    if row["Protein"] in top15_proteins:
        texts.append(
            plt.text(
                row["Log2FC"],
                -np.log10(row["adj_p_value"]),
                row["Protein"],
                fontsize=9
            )
        )

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black', lw=0.5))

plt.xlabel("log2 Fold Change (Responder / Non-responder)")
plt.ylabel("-log10(Adjusted p-value)")
plt.title("Volcano Plot (1.2-fold cutoff, adjusted p=0.05)")

plt.tight_layout()
plt.savefig("volcano_1.2fold_top15_labeled.png", dpi=300)
plt.show()
